The **Machine Learning Model Deployment** project will demonstrate the proficiency in using a Jupyter Notebook to take a machine learning model from training to deployment. The following actions will be involved in this project:

**Data Preprocessing:** Load and clean the data.

**Model Training:** Train a machine learning model on the processed data.

**Model Serialization:** Save the trained model so that it can be used later for predictions.

**Model Deployment:** Create a simple web API using Flask within the Jupyter Notebook to serve predictions.


In [ ]:
# Install necessary libraries

!pip install flask flask-ngrok scikit-learn

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok


In [ ]:
# Load dataset
iris = load_iris()
X = iris.data
y = iris.target

# Spliting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Displaying the shape of the dataset
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

In [ ]:
# Initialize model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy:.2f}")


In [ ]:
# Saving trained model
model_filename = 'iris_model.pkl'
joblib.dump(model, model_filename)
print(f"Model saved to {model_filename}")

In [ ]:
# Model Deployment!
# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Use ngrok to allow external access to the Flask app

# Load the model
model = joblib.load(model_filename)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    input_data = np.array(data['data'])
    prediction = model.predict(input_data)
    output = {'prediction': prediction.tolist()}
    return jsonify(output)

# Run app
if __name__ == '__main__':
    app.run()

In [ ]:
# Testing the API

import requests

# Sample input data
sample_data = {
    'data': [[5.1, 3.5, 1.4, 0.2]]
}

# Sending POST request to the API
response = requests.post('http://localhost:5000/predict', json=sample_data)

# Display response
print(response.json())
